# Лабораторная 4 - text

In [41]:
import pandas as pd
import numpy as np
import re
from transformers import pipeline
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

In [11]:
df = pd.read_csv('/content/scoutieDataset_81 (1).csv')
data = df[['text', 'views']]
data

,text,views
0,🔮\nНосить шубу из искусственного меха — значит...,29800
1,🟣\nПозаботьтесь о коже лица с уходовой космети...,21400
2,"💜\nДарим подарки, не дожидаясь Нового года!\nП...",9000
3,В списке новогодних хотелок твоей подруги или ...,54400
4,Делимся новостями! Наталья Шик представила сво...,40100
...,...,...
20385,"Когда-то Глюк’оZа появилась на наших экранах, ...",155
20386,Salvatore Ferragamo Emozione Dolce Fiore Остор...,150
20387,Палитра красных оттенков помад Color Riche Lip...,247
20388,"Набор Korres для тех, кто хочет дарить красоту...",157


# Очистка текстов

In [12]:
def clean_text(text, remove_emojis=False, remove_punctuation=True):
    text = text.replace("\n", " ")

    text = re.sub(r'\s+', ' ', text).strip()

    if remove_emojis:
        text = re.sub(r'[^\w\s,.!?]', '', text)

    if remove_punctuation:
        text = re.sub(r'[^\w\s]', '', text)

    return text

data = data.copy()
data['cleaned_text'] = data['text'].apply(lambda x: clean_text(x, remove_emojis=True, remove_punctuation=True))
data = data[['cleaned_text']]
data

,cleaned_text
0,Носить шубу из искусственного меха значит бы...
1,Позаботьтесь о коже лица с уходовой косметико...
2,Дарим подарки не дожидаясь Нового года Почувс...
3,В списке новогодних хотелок твоей подруги или ...
4,Делимся новостями Наталья Шик представила свой...
...,...
20385,Когдато ГлюкоZа появилась на наших экранах как...
20386,Salvatore Ferragamo Emozione Dolce Fiore Остор...
20387,Палитра красных оттенков помад Color Riche Lip...
20388,Набор Korres для тех кто хочет дарить красоту ...


In [38]:
data.to_csv("cleaned_dataset.csv", index=True)

# Таргет, промпт, разметка

**Таргет**: столбец "category", в котором могут содержаться 4 вида категорий, на которые можно классифицировать текст: "Реклама", "Товар", "Новости", "Общение"

Ты помощник для анализа текстов телеграмм-постов. Твоя задача — классифицировать тексты по следующим категориям:
1. "Реклама" — посты, в которых предлагается купить товар, есть скидки или рекламируется услуга.
2. "Товар" — посты, в которых описываются характеристики товаров или услуги без явного призыва к покупке.
3. "Новости" — посты, содержащие новости магазина или анонсы событий.
4. "Общение" — посты, которые содержат обращение к клиентам, вопросы или ответы на вопросы.

Верни только название категории для каждого текста.
Пример входного текста: "У нас скидка 20% на все летние коллекции! Успей купить до конца недели."
Пример ответа: "Реклама"


Но **я решила разметить вручную**, тк не нашла LLM, которая давала бы нужный результат

In [13]:
def classify_text_with_rules(text):
    if re.search(r"(конкурс|конкурса|начинаем|реклама|скидкаМскидки|купить|распродажа|акция|цена|дарим|розыгрыш|подборка|приходите)", text, re.IGNORECASE):
        return "Реклама"

    elif re.search(r"(состав|материал|функции|характеристики|удобный|практичный|сейчас|подарочный|набор)", text, re.IGNORECASE):
        return "Товар"

    elif re.search(r"(новости|анонс|обновление|событие|скоро)", text, re.IGNORECASE):
        return "Новости"

    elif re.search(r"(вопрос|ответ|обратная связь|как вам|помощь|пишите|рассказывайте)", text, re.IGNORECASE):
        return "Общение"

subset_data = data.head(8000)

subset_data["category"] = subset_data["cleaned_text"].apply(classify_text_with_rules)

subset_data.to_csv("labeled_dataset.csv", index=False)
print("завершено")

завершено


<ipython-input-13-ca452db528d4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data["category"] = subset_data["cleaned_text"].apply(classify_text_with_rules)


# Реализация текстовой модели: nn.Embedding + nn.LSTM + nn.Linear

vocab_size: количество уникальных слов

embed_size: размерность эмбеддингов

hidden_size: количество нейронов в скрытом слое LSTM

output_size: количество классов для классификации (в моем случае 4)

In [18]:
class TelegramTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size):
        super(TelegramTextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        _, (hn, _) = self.lstm(x)
        exit = self.fc(hn[-1])
        return exit

# Обучение модели

!!! **Цель тестирования — использовать модель для новых (нераспознанных) данных**

В этом случае тестовый набор содержит только тексты, и модель сама присваивает метки.

In [22]:
labeled_data = subset_data[subset_data['category'].notnull()]
unlabeled_data = subset_data[subset_data['category'].isnull()]
print(len(unlabeled_data))

# Преобразуем категории в числовые метки
label_to_idx = {label: idx for idx, label in enumerate(labeled_data['category'].unique())}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}
labeled_data['category'] = labeled_data['category'].map(label_to_idx)

train_texts = labeled_data['cleaned_text'].tolist()
train_labels = labeled_data['category'].tolist()

# !Выбираем подмножество неразмеченных данных для теста! Именно неразмеченных (по задумке)
test_texts = unlabeled_data['cleaned_text'].sample(n=3341, random_state=42).tolist()

# Создадим словарь для токенизации
vocab = {}
for text in train_texts:
    for word in text.split():
        if word not in vocab:
            vocab[word] = len(vocab)

class TelegramTextDataset(Dataset):
    def __init__(self, texts, targets, vocab):
        self.texts = texts
        self.targets = targets
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        text_indices = [self.vocab.get(word, 0) for word in text.split()]
        text_tensor = torch.tensor(text_indices, dtype=torch.long)

        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.long)
        else:
            target = -1
        return text_tensor, target

# DataLoader для обучения
train_dataset = TelegramTextDataset(train_texts, train_labels, vocab)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: collate_fn(x, pad_idx=0))

# DataLoader для тестовых данных
test_dataset = TelegramTextDataset(test_texts, targets=None, vocab=vocab)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: collate_fn(x, pad_idx=0))

# паддинг батчей
def collate_fn(batch, pad_idx=0):
    texts, targets = zip(*batch)
    max_len = max(len(text) for text in texts)
    padded_texts = torch.zeros((len(texts), max_len), dtype=torch.long) + pad_idx
    for i, text in enumerate(texts):
        padded_texts[i, :len(text)] = text
    targets = torch.tensor(targets, dtype=torch.long)
    return padded_texts, targets

model = TelegramTextClassificationModel(vocab_size=len(vocab) + 1, embed_size=100, hidden_size=128, output_size=len(label_to_idx))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучаем
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for texts, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

# Предсказание на тестовых данных
model.eval()
predictions = []
with torch.no_grad():
    for texts, _ in test_loader:
        outputs = model(texts)
        predicted_labels = torch.argmax(outputs, dim=1)
        predictions.extend(predicted_labels.tolist())

# предсказания - в текстовые метки
predicted_categories = [idx_to_label[idx] for idx in predictions]

unlabeled_sample = unlabeled_data.sample(n=3341, random_state=42)
unlabeled_sample['predicted_category'] = predicted_categories
unlabeled_sample.to_csv("test_predictions.csv", index=False)

3341


<ipython-input-22-acc6c082c1d9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_data['category'] = labeled_data['category'].map(label_to_idx)


Epoch [1/10], Loss: 103.1898
Epoch [2/10], Loss: 99.0666
Epoch [3/10], Loss: 88.9369
Epoch [4/10], Loss: 63.0688
Epoch [5/10], Loss: 52.3663
Epoch [6/10], Loss: 42.8721
Epoch [7/10], Loss: 38.4074
Epoch [8/10], Loss: 35.8522
Epoch [9/10], Loss: 31.5934
Epoch [10/10], Loss: 31.1942


# Оценка качества обучения

Функция для вычисления метрик на выборке

In [36]:
def evaluate_model(model, data_loader, criterion, idx_to_label=None):
    model.eval()
    all_targets = []
    all_predictions = []
    total_loss = 0

    with torch.no_grad():
        for texts, targets in data_loader:
            outputs = model(texts)
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_predictions.extend(preds.tolist())
            all_targets.extend(targets.tolist())

    labels = sorted(set(all_targets))

    accuracy = accuracy_score(all_targets, all_predictions)
    report = classification_report(
        all_targets,
        all_predictions,
        labels=labels,
        target_names=[idx_to_label[label] for label in labels] if idx_to_label else None,
    )
    cm = confusion_matrix(all_targets, all_predictions, labels=labels)

    return accuracy, total_loss / len(data_loader), report, cm

Так как в тестовой выберке у меня нет размеченных текстов, мы не можем посчитать классические метрики качества. Посчитаем только для train.

In [38]:
train_accuracy, train_loss, train_report, train_cm = evaluate_model(
    model, train_loader, criterion, idx_to_label
)
print("Train Metrics:")
print(f"Accuracy: {train_accuracy:.4f}, Loss: {train_loss:.4f}")
print("Classification Report:\n", train_report)
print("Confusion Matrix:\n", train_cm)

Train Metrics:
Accuracy: 0.8955, Loss: 0.2004
Classification Report:
               precision    recall  f1-score   support

     Общение       1.00      0.00      0.01       339
       Товар       0.46      0.98      0.62       410
     Реклама       1.00      1.00      1.00      3772
     Новости       0.00      0.00      0.00       138

    accuracy                           0.90      4659
   macro avg       0.61      0.49      0.41      4659
weighted avg       0.92      0.90      0.86      4659

Confusion Matrix:
 [[   1  338    0    0]
 [   0  400   10    0]
 [   0    1 3771    0]
 [   0  138    0    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Не посчитается нормально
test_accuracy, test_loss, test_report, test_cm = evaluate_model(
    model, test_loader, criterion, idx_to_label
)
print("Test Metrics:")
print(f"Accuracy: {test_accuracy:.4f}, Loss: {test_loss:.4f}")
print("Classification Report:\n", test_report)
print("Confusion Matrix:\n", test_cm)

В данном случае одним из самых примитивных способов оценить качество модели на test - посмотреть labelled_dataset.csv и оценить адекватьность присужденных метрик.

# Обучение модели с другим разделением train-test

!!! **Цель тестирования - оценить качество модели**

Для оценки качества модели (точность, F1-score, precision, recall) тестовый датасет обязательно должен содержать размеченные тексты. Разделим датасет другим образом так, чтобы тестовые данные тоже содержали размеченные тексты.

In [42]:
labeled_data = subset_data[subset_data['category'].notnull()]
train_data, test_data = train_test_split(labeled_data, test_size=0.2, random_state=42, stratify=labeled_data['category'])

label_to_idx = {label: idx for idx, label in enumerate(train_data['category'].unique())}
idx_to_label = {idx: label for label, idx in label_to_idx.items()}
train_data['category'] = train_data['category'].map(label_to_idx)
test_data['category'] = test_data['category'].map(label_to_idx)

train_texts = train_data['cleaned_text'].tolist()
train_labels = train_data['category'].tolist()

test_texts = test_data['cleaned_text'].tolist()
test_labels = test_data['category'].tolist()

vocab = {}
for text in train_texts:
    for word in text.split():
        if word not in vocab:
            vocab[word] = len(vocab) + 1

class TelegramTextDataset(Dataset):
    def __init__(self, texts, targets, vocab):
        self.texts = texts
        self.targets = targets
        self.vocab = vocab

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        text_indices = [self.vocab.get(word, 0) for word in text.split()]
        text_tensor = torch.tensor(text_indices, dtype=torch.long)

        if self.targets is not None:
            target = torch.tensor(self.targets[idx], dtype=torch.long)
        else:
            target = -1
        return text_tensor, target

train_dataset = TelegramTextDataset(train_texts, train_labels, vocab)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=lambda x: collate_fn(x, pad_idx=0))

test_dataset = TelegramTextDataset(test_texts, test_labels, vocab)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=lambda x: collate_fn(x, pad_idx=0))

def collate_fn(batch, pad_idx=0):
    texts, targets = zip(*batch)
    max_len = max(len(text) for text in texts)
    padded_texts = torch.zeros((len(texts), max_len), dtype=torch.long) + pad_idx
    for i, text in enumerate(texts):
        padded_texts[i, :len(text)] = text
    targets = torch.tensor(targets, dtype=torch.long)
    return padded_texts, targets

model = TelegramTextClassificationModel(vocab_size=len(vocab) + 1, embed_size=100, hidden_size=128, output_size=len(label_to_idx))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for texts, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

model.eval()
all_predictions = []
all_targets = []

with torch.no_grad():
    for texts, targets in test_loader:
        outputs = model(texts)
        predicted_labels = torch.argmax(outputs, dim=1)
        all_predictions.extend(predicted_labels.tolist())
        all_targets.extend(targets.tolist())

Epoch [1/10], Loss: 82.0869
Epoch [2/10], Loss: 79.4530
Epoch [3/10], Loss: 78.9957
Epoch [4/10], Loss: 79.0747
Epoch [5/10], Loss: 68.5362
Epoch [6/10], Loss: 47.8679
Epoch [7/10], Loss: 37.2705
Epoch [8/10], Loss: 30.5924
Epoch [9/10], Loss: 28.0187
Epoch [10/10], Loss: 25.0658


Значение функции потерь уменьшилось

# Снова оценка качества

Оценка модели на тренировочных данных

In [44]:
precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='macro')
accuracy = accuracy_score(all_targets, all_predictions)

print("Test Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

all_predictions_train = []
all_targets_train = []

model.eval()
with torch.no_grad():
    for texts, targets in train_loader:
        outputs = model(texts)
        predicted_labels = torch.argmax(outputs, dim=1)
        all_predictions_train.extend(predicted_labels.tolist())
        all_targets_train.extend(targets.tolist())

precision_train, recall_train, f1_train, _ = precision_recall_fscore_support(all_targets_train, all_predictions_train, average='macro')
accuracy_train = accuracy_score(all_targets_train, all_predictions_train)

print("Train Metrics:")
print(f"Accuracy: {accuracy_train:.4f}")
print(f"Precision: {precision_train:.4f}")
print(f"Recall: {recall_train:.4f}")
print(f"F1-score: {f1_train:.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Metrics:
Accuracy: 0.8401
Precision: 0.4620
Recall: 0.4362
F1-score: 0.4172
Train Metrics:
Accuracy: 0.9195
Precision: 0.5460
Recall: 0.6076
F1-score: 0.5731


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Оценка модели на тестовых данных

In [45]:
model.eval()
all_predictions = []
all_targets = []

with torch.no_grad():
    for texts, targets in test_loader:
        outputs = model(texts)
        predicted_labels = torch.argmax(outputs, dim=1)
        all_predictions.extend(predicted_labels.tolist())
        all_targets.extend(targets.tolist())

precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_predictions, average='macro')
accuracy = accuracy_score(all_targets, all_predictions)

print("Test Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Test Metrics:
Accuracy: 0.8401
Precision: 0.4620
Recall: 0.4362
F1-score: 0.4172


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
